In [3]:
# Загружаем бибилиотеки
import pandas as pd
from urllib.parse import quote_plus
from sqlalchemy import create_engine
from sqlalchemy.orm.session import sessionmaker

In [4]:
# Создаём подключение
conn_string = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=192.168.189.71;DATABASE=Dokuchaev;UID=sa;PWD=As12345^'
quoted = quote_plus(conn_string)
new_conn_string = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(new_conn_string)

In [6]:
%%time
sales = pd.read_sql('SELECT * FROM dbo.sales_tb', engine)

Wall time: 2.6 s


In [8]:
sales.head(5)

,Store.Id,name,year,month,orders,sales,discount,type,code
0,dc590a6f-a302-4883-af69-8d612e273130,Лесная 43_СТ_МСК,2020,1,175,27060.0,360.0,STORE,1221-12
1,277334f1-0d3c-4818-a934-81641a83c2bf,Новый Арбат 3_СТ_МСК,2020,1,379,55440.0,960.0,STORE,2125
2,c27d8f86-50b9-49be-9abe-7a9b8ebc8fe3,Архив Метрополис 1_ТЦ_МСК,2018,8,638,86980.0,1260.0,STORE,12
3,bc5d3170-e307-468a-9437-0609f3bcf91d,Павелецкий_ТХ_МСК,2018,8,728,92000.0,1980.0,STORE,110
4,681f37cd-1166-4ead-a7b9-6c29fd4237b4,Свиблово_ТРЦ_МСК,2019,9,386,51000.0,1320.0,STORE,97


In [24]:
sales[['name','year', 'month', 'sales']]

,name,year,month,sales
0,Лесная 43_СТ_МСК,2020,1,27060.0
1,Новый Арбат 3_СТ_МСК,2020,1,55440.0
2,Архив Метрополис 1_ТЦ_МСК,2018,8,86980.0
3,Павелецкий_ТХ_МСК,2018,8,92000.0
4,Свиблово_ТРЦ_МСК,2019,9,51000.0
...,...,...,...,...
93588,Невский пр 11_СТ_СПБ,2020,1,57660.0
93589,(АРХИВ)Боткинская 3_ТЦ_СПБ,2018,12,18340.0
93590,Покровка 14_СТ_МСК,2018,6,63260.0
93591,Пятницкая 31/2_СТ_МСК,2018,11,58840.0


In [32]:
sales = sales[['name','year', 'month', 'sales']].groupby(by=['name','year', 'month'], as_index=False).sum()

In [33]:
sales[['year', 'month']].drop_duplicates().sort_values(by = ['year','month'])

,year,month
18,2018,1
19,2018,2
20,2018,3
21,2018,4
22,2018,5
23,2018,6
24,2018,7
0,2018,8
1,2018,9
2,2018,10


In [34]:
sales = sales[(sales.year <= 2019) | ((sales.year == 2020) & (sales.month <= 3))]

In [35]:
sales[['year', 'month']].drop_duplicates().sort_values(by = ['year','month'])

,year,month
18,2018,1
19,2018,2
20,2018,3
21,2018,4
22,2018,5
23,2018,6
24,2018,7
0,2018,8
1,2018,9
2,2018,10


In [41]:
opened_1q_2018 = sales[(sales.year == 2018) & (sales.month <= 3)].name.unique()
print(opened_1q_2018, len(opened_1q_2018))

['(Архив 28.06)Город Лефортово_ТЦ_МСК' '(Архив) Радуга_ТЦ_СПБ'
 '(Архив)Золотой Вавилон Отрадное_ТЦ_МСК' '(Архив)Кофейня Садовая 19'
 'Авиапарк 3_ТЦ_МСК' 'Архив  Сокольники' 'Архив Авеню_ТЦ_МСК'
 'Архив Арбат' 'Архив Багратионовская' 'Архив Бауманская' 'Архив Водный'
 'Архив Гагаринский_ТЦ_МСК' 'Архив Золотой Вавилон Ростокино'
 'Архив Ленинградский' 'Архив Люсиновская' 'Архив Метрополис 1_ТЦ_МСК'
 'Архив Планерная' 'Архив Сусальный Н 5_СТ_МСК' 'Архив ТЦ Щука'
 'Архив Тверской' 'Архив мякинино' 'Архив с 01/06 МДМ_ТЦ_МСК_'
 'Архив(01.07)Калейдоскоп_ТЦ_МСК' 'Архив(01.09)Мега Химки_ТЦ_МО'
 'Архив(01.10)Мега Белая Дача_ТЦ_МО' 'АрхивТЦ Калужский '
 'Афимолл 1_ТЦ_МСК' 'Вегас Каширка_ТЦ_МСК' 'Зеленопарк_ТЦ_МО'
 'Инвентарь Одоевского 31' 'Колумбус 1_ТЦ_МСК' 'Красный Кит_ТЦ_МО'
 'Кузнецкий Мост 12_СТ_МСК' 'Ленинградский 60_СТ_МСК' 'Мозаика_ТЦ_МСК'
 'Невский пр 11_СТ_СПБ' 'Невский пр 81_СТ_СПБ' 'Новомосковский_ТЦ_МО'
 'Одоевского 31_СТ_СПБ' 'Охотный Ряд_1_ТЦ_МСК' 'Павелецкий_ТХ_МСК'
 'Перловский

In [ ]:
woring_24m